# Module

In [1]:
import torch
from PIL import Image
from transformers import VisionEncoderDecoderModel, AutoTokenizer, TrOCRProcessor
from glob import glob
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import json
import os

from ultralytics import YOLO

/root/miniconda3/envs/trocr/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 장치

In [2]:
device = torch.device('cuda3')

# 모형

In [40]:
# 사전 학습 모형 불러오기
model = VisionEncoderDecoderModel.from_pretrained("team-lucid/trocr-small-korean").to(device)

In [41]:
# 전처리 도구
processor = TrOCRProcessor.from_pretrained("team-lucid/trocr-small-korean")
tokenizer = AutoTokenizer.from_pretrained('team-lucid/trocr-small-korean')

In [1]:
# YOLO 사용 시 불러오기
#yolo = YOLO('/workspace/ojt/OCR/YOLO/runs/detect/best_yolov8n/weights/best.pt').to(device)

In [6]:
# 학습 가중치 불러오기
model.load_state_dict(torch.load('/workspace/ojt/OCR/TrOCR/save_point_28.pth'))

<All keys matched successfully>

# 추론

In [7]:
# 추론할 사진 폴더 
imgs = sorted(glob('/workspace/ojt/OCR/dataset/infer_final/image/*'))

In [8]:
imgs

['/workspace/ojt/OCR/dataset/infer_final/image/008b1a0b-2a55-2054-f3bb-fd20f5b69628.jpg',
 '/workspace/ojt/OCR/dataset/infer_final/image/0de9fdab-1873-a95d-7ebe-19aae551a32d.jpg',
 '/workspace/ojt/OCR/dataset/infer_final/image/1d99d954-8082-594d-4cb1-eba581e8caef.jpg',
 '/workspace/ojt/OCR/dataset/infer_final/image/bd036c86-e028-cdbd-e06a-a37f1162816d.jpg',
 '/workspace/ojt/OCR/dataset/infer_final/image/db4aac6f-0810-0df6-92fa-62cbd1b7cebb.jpg']

In [55]:
# YOLO를 사용할 경우 YOLO 결과
#results = yolo(imgs[2])


image 1/1 /workspace/ojt/OCR/dataset/infer/imgs/sample3.png: 2048x800 76 Texts, 72.0ms
Speed: 7.4ms preprocess, 72.0ms inference, 1.4ms postprocess per image at shape (1, 3, 2048, 800)


In [22]:
# 예측 결과 불러오기
bboxes = sorted(glob('/workspace/ojt/OCR/CRAFT-pytorch/result/*txt'))[:5]
bboxes

['/workspace/ojt/OCR/CRAFT-pytorch/result/res_008b1a0b-2a55-2054-f3bb-fd20f5b69628.txt',
 '/workspace/ojt/OCR/CRAFT-pytorch/result/res_0de9fdab-1873-a95d-7ebe-19aae551a32d.txt',
 '/workspace/ojt/OCR/CRAFT-pytorch/result/res_1d99d954-8082-594d-4cb1-eba581e8caef.txt',
 '/workspace/ojt/OCR/CRAFT-pytorch/result/res_bd036c86-e028-cdbd-e06a-a37f1162816d.txt',
 '/workspace/ojt/OCR/CRAFT-pytorch/result/res_db4aac6f-0810-0df6-92fa-62cbd1b7cebb.txt']

In [74]:
import time
start = time.time()
#results = yolo(imgs[2])
with open(bboxes[4], 'r') as f:
    boxes = f.readlines()
boxes = list(map(lambda x: x.strip(), boxes))
#boxes = results[0].boxes.xyxy.cpu().numpy()
sample = cv2.imread(imgs[4])
sample = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)
print('경계 상자 개수 : ',len(boxes))
patches = []
texts = []
for j in range(len(boxes)):
    x = boxes[j].split(',')
    x = np.array(x,dtype=np.int32)
    xs = x[1::2]
    ys = x[::2]
    min_x = np.min(xs)
    max_x = np.max(xs)
    min_y = np.min(ys)
    max_y = np.max(ys)
    patch = sample[min_x : max_x + 1, min_y : max_y + 1,:]
    patches.append([min_x,min_y,max_x,max_y])
    img = Image.fromarray(patch)
    pixel_values =(processor(img,return_tensors='pt').pixel_values).to(device)
    generated_ids1 = model.generate(pixel_values)
    generated_text1 = tokenizer.batch_decode(generated_ids1, skip_special_tokens=True)[0]
    texts.append(generated_text1)
print(time.time()-start)
    

경계 상자 개수 :  677


/root/miniconda3/envs/trocr/lib/python3.8/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


56.309916257858276


# 결과 저장

In [46]:
import cv2
import numpy as np
import platform
from PIL import ImageFont, ImageDraw, Image
import matplotlib.pyplot as plt
plt.rc('font', family='/workspace/ojt/OCR/TrOCR/malgun.ttf')

In [46]:
pixel_values =(processor(img,return_tensors='pt').pixel_values).to(device)
generated_ids1 = model.generate(pixel_values)
generated_text1 = tokenizer.batch_decode(generated_ids1, skip_special_tokens=True)[0]
generated_ids2 = model2.generate(pixel_values)
generated_text2 = tokenizer.batch_decode(generated_ids2, skip_special_tokens=True)[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
print(generated_text1)
print(generated_text2)
plt.imshow(np.array(img))
plt.show()

In [75]:
from PIL import Image, ImageDraw, ImageFont

image = Image.open(imgs[4]).convert('RGB')
# 이미지 크기 설정 (예: 800x600)
image_width = image.size[0]
image_height = image.size[1]

# 주어진 좌표와 텍스트 정보

for i in range(len(patches)):
    min_x, min_y, max_x, max_y = patches[i][1], patches[i][0], patches[i][3], patches[i][2]
    #text = texts[i]
    text = texts[i]

    # 이미지 생성 (흰색 배경)

    draw = ImageDraw.Draw(image)

    # 텍스트를 그릴 폰트 설정 (기본 폰트 사용)
    font = ImageFont.truetype('/workspace/ojt/OCR/TrOCR/malgun.ttf',size=50)
    draw.rectangle([min_x, min_y, max_x, max_y], outline="green", width=10)
    # 텍스트 크기 계산
    bbox = draw.textbbox((0, 0), text, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]

    # 텍스트를 그릴 위치 계산 (중앙 정렬)
    text_x = min_x + (max_x - min_x - text_width) / 2
    text_y = min_y - 30

    # 텍스트 그리기
    draw.text((text_x, text_y), text, fill="blue", font=font,fontsize=10)

    # 이미지 저장
image.save("output_image3.png")

# 이미지 보기 (선택 사항)
image.show()

display-im6.q16: unable to open X server `' @ error/display.c/DisplayImageCommand/412.


In [6]:
texts_path = glob('/workspace/ojt/OCR/dataset/infer_final/text_files/*')
len(texts_path)

1966

In [9]:
import pandas as pd
df = pd.read_csv('/workspace/ojt/OCR/dataset/infer_final/final_datset.csv')
df.head()

,id,img_path,label
0,TRAIN_1,./train/TRAIN_1.jpg,식품의
1,TRAIN_2,./train/TRAIN_2.jpg,유형
2,TRAIN_3,./train/TRAIN_3.jpg,커피
3,TRAIN_4,./train/TRAIN_4.jpg,유통기한
4,TRAIN_5,./train/TRAIN_5.jpg,용기


In [17]:
preds = []
for path in tqdm(df.img_path):
    new_path = path.replace('./train', '/workspace/ojt/OCR/dataset/infer_final/text_files')
    img = Image.open(new_path).convert('RGB')
    pixel_values =(processor(img,return_tensors='pt').pixel_values).to(device)
    generated_ids1 = model.generate(pixel_values)
    generated_text1 = tokenizer.batch_decode(generated_ids1, skip_special_tokens=True)[0]
    preds.append(generated_text1)

  0%|          | 0/1966 [00:00<?, ?it/s]/root/miniconda3/envs/trocr/lib/python3.8/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1966/1966 [02:44<00:00, 11.93it/s]


# 정확도 계산

In [18]:
df['preds'] = preds 
df.tail()

,id,img_path,label,preds
1961,TRAIN_1962,./train/TRAIN_1962.jpg,영양성분,영양성분
1962,TRAIN_1963,./train/TRAIN_1963.jpg,기준치에,기준치에
1963,TRAIN_1964,./train/TRAIN_1964.jpg,및,및
1964,TRAIN_1965,./train/TRAIN_1965.jpg,또는,또는
1965,TRAIN_1966,./train/TRAIN_1966.jpg,수,수


In [19]:
c = 0
for i in range(len(df)):
    if df.loc[i, 'label'].replace(' ','') == df.loc[i, 'preds'].replace(' ',''):
        c += 1 

In [14]:
c / len(df)

0.3387589013224822

In [20]:
c / len(df)

0.6673448626653102

In [ ]:
# 틀린 결과 보기
df[df.label != df.preds].head(20)